In [ ]:
!git clone https://github.com/Latortuga13/CSE151Diabetes.git

from re import A
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

#accessing our data
data = pd.read_excel("CSE151Diabetes/Patients Data ( Used for Heart Disease Prediction ).xlsx")


fatal: destination path 'CSE151Diabetes' already exists and is not an empty directory.
<class 'pandas.core.frame.DataFrame'>
Index: 2376 entries, 39973 to 32459
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Sex                        2376 non-null   object 
 1   GeneralHealth              2376 non-null   object 
 2   AgeCategory                2376 non-null   object 
 3   BMI                        2376 non-null   float64
 4   HadHeartAttack             2376 non-null   int64  
 5   HadAngina                  2376 non-null   int64  
 6   HadStroke                  2376 non-null   int64  
 7   HadAsthma                  2376 non-null   int64  
 8   HadSkinCancer              2376 non-null   int64  
 9   HadCOPD                    2376 non-null   int64  
 10  HadDepressiveDisorder      2376 non-null   int64  
 11  HadKidneyDisease           2376 non-null   int64  
 12  HadArthritis     

In [ ]:
# dropping columns we discussed in the README.
data_clean = data.drop(columns=["PatientID", 'HeightInMeters', "WeightInKilograms", "State", "SmokerStatus", "ECigaretteUsage", "TetanusLast10Tdap"])
dSample = data_clean.sample(frac=0.01, random_state=42, axis=0)
dSample.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2376 entries, 39973 to 32459
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Sex                        2376 non-null   object 
 1   GeneralHealth              2376 non-null   object 
 2   AgeCategory                2376 non-null   object 
 3   BMI                        2376 non-null   float64
 4   HadHeartAttack             2376 non-null   int64  
 5   HadAngina                  2376 non-null   int64  
 6   HadStroke                  2376 non-null   int64  
 7   HadAsthma                  2376 non-null   int64  
 8   HadSkinCancer              2376 non-null   int64  
 9   HadCOPD                    2376 non-null   int64  
 10  HadDepressiveDisorder      2376 non-null   int64  
 11  HadKidneyDisease           2376 non-null   int64  
 12  HadArthritis               2376 non-null   int64  
 13  HadDiabetes                2376 non-null   objec

In [ ]:
#cleaning our data
data_clean["AgeCategory"] = data["AgeCategory"].str.extract(r"(\d+)")
data_clean["AgeCategory"] = data_clean["AgeCategory"].astype(int)

data_clean = pd.get_dummies(data_clean, columns=['RaceEthnicityCategory'])

health_mapping = {
    "Excellent": 5,
    "Very good": 4,
    "Good": 3,
    "Fair": 2,
    "Poor": 1
}
data_clean["GeneralHealth"] = data_clean["GeneralHealth"].map(health_mapping)
data_clean["GeneralHealth"] = data_clean["GeneralHealth"].astype(int)

sex_mapping = {
    "Male": 1,
    "Female": 0
}

data_clean["Sex"] = data_clean["Sex"].map(sex_mapping)
data_clean["Sex"] = data_clean["Sex"].astype(int)

data_clean.head()


,Sex,GeneralHealth,AgeCategory,BMI,HadHeartAttack,HadAngina,HadStroke,HadAsthma,HadSkinCancer,HadCOPD,...,HIVTesting,FluVaxLast12,PneumoVaxEver,HighRiskLastYear,CovidPos,"RaceEthnicityCategory_Black only, Non-Hispanic",RaceEthnicityCategory_Hispanic,"RaceEthnicityCategory_Multiracial, Non-Hispanic","RaceEthnicityCategory_Other race only, Non-Hispanic","RaceEthnicityCategory_White only, Non-Hispanic"
0,0,2,75,32.099998,0,1,0,1,1,0,...,0,0,1,0,1,False,False,False,False,True
1,0,4,65,27.990000,0,0,0,0,0,0,...,0,1,1,0,0,False,False,False,False,True
2,1,5,60,22.530001,0,0,0,0,0,0,...,0,0,0,0,0,False,False,False,False,True
3,1,4,70,30.129999,0,0,0,0,0,0,...,0,1,1,0,0,False,False,False,False,True
4,0,3,50,27.760000,0,0,0,0,0,0,...,0,1,0,0,0,True,False,False,False,False


In [ ]:
#Scaled the data to make it easier to compare the weights generated from our model for each feature afterwards.

from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()

age = pd.DataFrame(data_clean, columns=['AgeCategory'])
age = min_max_scaler.fit_transform(age)
data_clean['AgeCategory'] = age

bmi = pd.DataFrame(data_clean, columns=['BMI'])
bmi = min_max_scaler.fit_transform(bmi)
data_clean['BMI'] = bmi
data_clean.head()


In [ ]:
#Splitting data to run model
from sklearn.model_selection import train_test_split

X = data_clean.drop("HadDiabetes", axis=1)
Y = data_clean['HadDiabetes']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(190104, 31)
(47526, 31)
(190104,)
(47526,)


In [ ]:
#running of model without cleaning the HadDiabetes column
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



Classification Report:


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


                                         precision    recall  f1-score   support

                                     No       0.85      0.97      0.91     39410
No, pre-diabetes or borderline diabetes       0.00      0.00      0.00      1068
                                    Yes       0.50      0.19      0.28      6667
Yes, but only during pregnancy (female)       0.00      0.00      0.00       381

                               accuracy                           0.83     47526
                              macro avg       0.34      0.29      0.30     47526
                           weighted avg       0.77      0.83      0.79     47526



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
#running of cleaned HadDiabetes Column along with a model using only the test data
data_clean = data_clean[data_clean["HadDiabetes"] != "Yes, but only during pregnancy (female)"]
data_clean = data_clean[data_clean["HadDiabetes"] != "No, pre-diabetes or borderline diabetes"]

X = data_clean.drop("HadDiabetes", axis=1)
Y = data_clean['HadDiabetes']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

y_pred2 = model.predict(X_train)

print("\nClassification Report:")
print(classification_report(y_train, y_pred2))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



Classification Report:
              precision    recall  f1-score   support

          No       0.87      0.98      0.92     39450
         Yes       0.55      0.17      0.26      6654

    accuracy                           0.86     46104
   macro avg       0.71      0.57      0.59     46104
weighted avg       0.83      0.86      0.83     46104


Classification Report:
              precision    recall  f1-score   support

          No       0.88      0.98      0.92    158013
         Yes       0.55      0.17      0.26     26401

    accuracy                           0.86    184414
   macro avg       0.71      0.57      0.59    184414
weighted avg       0.83      0.86      0.83    184414



In [ ]:
#Running a model with a vastly different test size
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.9999, random_state=0)

model = LogisticRegression()
model.fit(X_train, y_train)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

y_pred = model.predict(X_test)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(23, 31)
(230495, 31)
(23,)
(230495,)

Classification Report:
              precision    recall  f1-score   support

          No       0.86      0.99      0.92    197442
         Yes       0.40      0.03      0.05     33053

    accuracy                           0.85    230495
   macro avg       0.63      0.51      0.49    230495
weighted avg       0.79      0.85      0.80    230495



In [ ]:
#running a model with oversampled Yes data to have a more balanced data set
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

model = LogisticRegression()
model.fit(X_resampled, y_resampled)

y_pred = model.predict(X_test)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

y_pred2 = model.predict(X_train)

print("\nClassification Report:")
print(classification_report(y_train, y_pred2))


Classification Report:
              precision    recall  f1-score   support

          No       0.95      0.74      0.83     39489
         Yes       0.32      0.75      0.45      6615

    accuracy                           0.74     46104
   macro avg       0.63      0.74      0.64     46104
weighted avg       0.86      0.74      0.77     46104


Classification Report:
              precision    recall  f1-score   support

          No       0.95      0.73      0.83    157974
         Yes       0.32      0.75      0.45     26440

    accuracy                           0.74    184414
   macro avg       0.63      0.74      0.64    184414
weighted avg       0.86      0.74      0.77    184414



##Where does your model fit in the fitting graph? and What are the next models you are thinking of and why?

**The predictive error between the test data and train data is extremely close (even given how many entries given in our dataset). Therefore, we believe our model is underfitted. The next model we are thinking of is an SVM model. SVM's generally create a more complex decision boundary, and because we are underfitting we believe this will help us produce a more generalizable model.**